# Testing SDTW

In [3]:
import numpy as np
import os 
import torch

from lib.nn_losses.soft_dtw_cuda import SoftDTW 

In [13]:
x = [[4.8994e-07, 4.5955e-04, 1.8860e-02, 1.0195e-08, 6.6955e-05, 1.5712e-04,
         2.2970e-06, 4.6404e-03, 1.0738e-07, 7.3090e-07, 9.7580e-01, 1.4782e-07,
         2.6065e-07, 3.0750e-06, 7.8884e-06, 1.9705e-07, 4.9155e-07, 6.4610e-07,
         4.2357e-07, 7.8589e-07, 2.0028e-07, 2.1902e-07, 1.3477e-06, 1.2996e-06],
        [7.5114e-04, 7.3236e-03, 5.2179e-01, 2.8828e-04, 6.2930e-03, 3.0842e-02,
         1.9267e-03, 6.8575e-02, 3.5613e-03, 4.1769e-04, 3.5359e-01, 2.0815e-03,
         4.6765e-05, 1.4315e-04, 5.4171e-04, 1.3669e-04, 1.6447e-04, 2.6654e-04,
         2.6593e-04, 4.0475e-04, 2.5311e-04, 1.1573e-04, 1.0819e-04, 1.0609e-04],
        [2.4165e-04, 4.4685e-03, 1.4582e-01, 2.6239e-05, 2.4955e-03, 2.7514e-03,
         1.6659e-03, 1.2441e-01, 1.5894e-05, 6.9969e-04, 7.1660e-01, 7.5114e-05,
         1.5933e-05, 6.9353e-05, 1.0741e-04, 1.4621e-05, 3.0423e-05, 4.4083e-05,
         9.6989e-05, 1.5428e-04, 4.1357e-05, 3.6135e-05, 6.9764e-05, 5.0658e-05],
        [4.4426e-09, 1.6269e-06, 2.1836e-02, 1.1904e-11, 7.8197e-06, 7.1400e-07,
         1.2123e-07, 6.6827e-04, 3.7749e-10, 5.0269e-09, 9.7749e-01, 3.6152e-10,
         1.0620e-09, 2.6545e-08, 1.8458e-07, 3.4253e-09, 1.5327e-08, 1.6168e-08,
         1.8439e-08, 5.1398e-08, 1.0144e-08, 1.0166e-08, 5.8718e-08, 2.3374e-08],
        [2.8440e-04, 6.8169e-02, 1.2248e-02, 2.3419e-05, 1.2466e-02, 3.7124e-03,
         2.9647e-04, 4.5379e-02, 6.0442e-05, 5.6094e-04, 8.5582e-01, 6.6839e-05,
         5.2370e-05, 2.4561e-04, 2.0264e-04, 2.3593e-05, 5.2015e-05, 4.2178e-05,
         2.0379e-05, 3.0980e-05, 1.5557e-05, 2.1769e-05, 1.0363e-04, 1.0056e-04],
        [2.2404e-07, 5.6291e-04, 9.2299e-04, 1.8801e-09, 4.2415e-05, 2.6821e-04,
         2.6017e-08, 9.7708e-04, 3.7125e-07, 7.8197e-09, 9.9722e-01, 3.1969e-08,
         2.1893e-07, 1.7625e-06, 3.9180e-06, 5.9276e-08, 2.3890e-07, 3.4877e-07,
         4.1274e-08, 7.8974e-08, 2.0282e-08, 1.8019e-08, 2.5977e-07, 6.3606e-07],
        [4.2342e-07, 1.6123e-04, 4.3614e-03, 2.4655e-09, 1.5012e-05, 7.5717e-04,
         7.1769e-08, 4.3405e-04, 3.0487e-07, 2.9210e-08, 9.9426e-01, 1.7992e-08,
         1.3799e-07, 1.0229e-06, 3.9633e-06, 8.0944e-08, 3.1281e-07, 5.2594e-07,
         8.1839e-08, 1.0358e-07, 2.7165e-08, 3.7874e-08, 3.0316e-07, 4.7875e-07],
        [6.3325e-02, 2.4307e-04, 2.4532e-01, 2.0249e-03, 7.7391e-04, 1.4343e-01,
         7.0972e-04, 1.8343e-03, 4.9367e-01, 6.3028e-05, 4.1541e-02, 4.5137e-03,
         5.3510e-04, 2.3128e-04, 6.6264e-04, 1.6800e-04, 8.4352e-05, 1.4243e-04,
         7.7364e-05, 8.6845e-05, 1.0973e-04, 8.9731e-05, 6.8734e-05, 2.9086e-04],
        [8.7672e-02, 1.8452e-04, 7.0212e-03, 5.8812e-03, 7.6622e-04, 1.4728e-01,
         5.1988e-04, 1.1389e-03, 6.0358e-01, 1.2895e-05, 1.3063e-01, 9.9568e-03,
         1.9916e-03, 9.3490e-05, 6.9633e-05, 3.1457e-05, 5.1952e-05, 1.8405e-04,
         5.6787e-05, 4.7016e-05, 4.4365e-05, 8.3753e-05, 3.6723e-04, 2.3340e-03],
        [6.3233e-05, 9.4420e-04, 9.1782e-05, 3.6820e-03, 1.7100e-04, 1.4554e-04,
         7.2784e-03, 1.8242e-04, 3.9620e-05, 2.9127e-04, 9.8572e-01, 3.1865e-05,
         9.4063e-06, 2.3017e-06, 2.5447e-06, 6.8136e-06, 1.3854e-05, 1.0146e-05,
         3.9638e-06, 4.1844e-06, 4.5077e-06, 7.9575e-05, 1.1129e-03, 1.0989e-04],
        [3.1075e-01, 1.6592e-04, 1.7613e-01, 2.8833e-03, 6.1041e-04, 2.7455e-01,
         3.0084e-04, 2.1131e-02, 4.3732e-02, 1.6055e-04, 1.6329e-01, 2.0286e-03,
         1.5036e-03, 3.7945e-04, 5.6028e-04, 9.0482e-05, 8.4091e-05, 1.1902e-04,
         8.0206e-05, 9.9124e-05, 6.4451e-05, 1.1252e-04, 1.6075e-04, 1.0137e-03],
        [6.2597e-02, 7.7685e-06, 9.0515e-03, 1.9521e-02, 2.2234e-06, 4.2934e-01,
         2.3997e-05, 2.8720e-04, 4.5102e-01, 5.3590e-07, 2.6866e-02, 6.5202e-04,
         2.6047e-04, 2.4696e-05, 1.2948e-04, 3.7514e-05, 1.5351e-05, 3.9550e-05,
         1.1482e-05, 1.4304e-05, 8.2610e-06, 9.5682e-06, 9.6363e-06, 7.4611e-05],
        [2.0849e-01, 8.9807e-06, 1.2791e-03, 7.9784e-03, 1.7459e-05, 8.7258e-02,
         2.4209e-04, 1.1159e-05, 6.9291e-01, 1.3669e-05, 4.3393e-04, 8.6632e-04,
         2.4555e-04, 1.7924e-05, 2.6068e-05, 2.2311e-05, 1.5299e-05, 5.2826e-05,
         2.3219e-05, 1.0548e-05, 9.2761e-06, 1.1415e-05, 6.9923e-06, 4.3923e-05],
        [2.2971e-01, 3.2546e-04, 6.7408e-02, 2.5158e-02, 2.5248e-03, 3.6871e-01,
         2.3679e-03, 7.4116e-03, 2.3851e-01, 1.2173e-03, 2.4907e-02, 2.6180e-02,
         1.0095e-03, 1.5633e-04, 2.6192e-04, 2.5171e-04, 3.7394e-04, 7.3851e-04,
         4.2781e-04, 2.7074e-04, 2.5249e-04, 3.7306e-04, 3.4993e-04, 1.1037e-03],
        [6.2217e-02, 1.1837e-08, 1.6767e-05, 2.1017e-03, 3.1918e-08, 2.8048e-02,
         7.2332e-07, 9.6488e-08, 9.0754e-01, 2.0813e-08, 4.9878e-05, 9.8930e-06,
         1.3751e-05, 1.3565e-07, 3.7378e-07, 3.6285e-07, 2.2383e-07, 1.0137e-06,
         2.4074e-07, 2.4633e-07, 4.4243e-07, 1.2193e-06, 5.4909e-07, 3.0383e-06],
        [1.5271e-01, 1.0062e-05, 5.4746e-03, 3.1344e-03, 1.3787e-04, 3.2101e-01,
         7.2143e-05, 2.1304e-04, 5.1072e-01, 2.4047e-05, 2.2965e-03, 3.2763e-03,
         2.8790e-04, 1.3724e-05, 2.0966e-05, 1.7666e-05, 3.3689e-05, 1.2785e-04,
         4.5952e-05, 2.6158e-05, 3.0582e-05, 4.6819e-05, 3.8603e-05, 2.2966e-04],
        [5.6744e-03, 1.3299e-04, 1.6635e-02, 9.4021e-03, 1.4769e-05, 7.2631e-01,
         1.1088e-04, 4.7239e-04, 1.5051e-01, 7.5147e-06, 8.8466e-02, 1.4802e-03,
         1.2716e-04, 2.1343e-05, 1.1606e-04, 5.4478e-05, 4.1397e-05, 1.3181e-04,
         3.1463e-05, 2.7063e-05, 2.2529e-05, 3.6745e-05, 4.9648e-05, 1.2383e-04],
        [2.2183e-02, 3.2125e-06, 1.3633e-03, 4.8380e-03, 1.3043e-06, 5.9097e-01,
         8.0805e-06, 1.8748e-05, 3.7590e-01, 3.5296e-07, 4.3518e-03, 1.8251e-04,
         4.9342e-05, 3.0001e-06, 1.6148e-05, 1.0373e-05, 1.0301e-05, 4.8003e-05,
         8.1296e-06, 4.8635e-06, 3.1626e-06, 4.3169e-06, 3.4770e-06, 1.8568e-05],
        [4.1889e-01, 3.0171e-03, 2.5528e-03, 2.1112e-01, 7.8217e-04, 1.8678e-01,
         7.2053e-03, 3.7022e-03, 2.4098e-02, 4.3881e-03, 1.3228e-01, 7.5860e-04,
         8.9039e-04, 2.1404e-04, 2.3829e-04, 2.9781e-04, 4.7491e-04, 5.8872e-04,
         2.7506e-04, 1.8670e-04, 7.9962e-05, 2.8424e-04, 4.3253e-04, 4.5548e-04],
        [9.3216e-05, 9.8758e-04, 3.0755e-05, 4.2008e-03, 3.5729e-06, 1.0044e-04,
         9.7487e-03, 5.7579e-05, 6.3007e-07, 3.5516e-04, 9.8433e-01, 1.0801e-07,
         1.5412e-06, 1.9080e-06, 3.3876e-06, 5.0924e-06, 6.8413e-06, 4.3946e-06,
         3.2023e-06, 2.6530e-06, 4.8588e-07, 7.7270e-06, 4.9355e-05, 2.8628e-06],
        [7.9142e-05, 4.6606e-04, 4.7724e-05, 2.2870e-03, 2.4347e-05, 1.3939e-04,
         7.8305e-03, 3.3392e-05, 1.6554e-05, 5.5269e-04, 9.8791e-01, 3.6377e-07,
         1.0768e-06, 5.0372e-07, 1.0208e-06, 2.9291e-06, 5.9796e-06, 4.8143e-06,
         3.2323e-06, 5.5495e-06, 5.8859e-06, 1.2686e-04, 4.4765e-04, 9.1527e-06],
        [3.2000e-02, 4.6638e-01, 1.0604e-03, 4.2665e-02, 6.2049e-03, 5.8237e-03,
         2.0291e-01, 4.3954e-03, 8.2812e-03, 1.3005e-01, 9.2003e-02, 6.4446e-04,
         9.5000e-04, 1.4021e-03, 3.8174e-04, 1.8156e-04, 9.9628e-05, 1.0499e-04,
         2.2326e-04, 5.3494e-04, 5.6788e-04, 1.2145e-03, 1.5126e-03, 4.0137e-04],
        [1.4295e-04, 1.0095e-01, 9.5887e-06, 8.9581e-04, 3.1739e-05, 1.7021e-04,
         5.0236e-03, 4.4454e-05, 7.2227e-06, 7.2425e-04, 8.9184e-01, 8.7809e-08,
         9.0952e-06, 3.0275e-05, 1.6432e-05, 5.7765e-06, 4.5861e-06, 2.9204e-06,
         1.5510e-06, 2.7774e-06, 1.1350e-06, 1.0509e-05, 6.6503e-05, 4.9954e-06],
        [2.0122e-05, 1.3210e-04, 1.0082e-06, 7.8562e-05, 1.1032e-07, 1.1117e-05,
         7.2537e-05, 2.7718e-06, 2.2280e-07, 6.4762e-07, 9.9968e-01, 1.3352e-10,
         1.1909e-07, 2.3615e-07, 4.6153e-07, 1.5736e-07, 1.6812e-07, 1.1213e-07,
         5.3700e-08, 1.4046e-07, 2.1210e-08, 2.2735e-07, 1.3050e-06, 7.6277e-08],
        [3.9293e-02, 5.5094e-02, 4.5323e-03, 2.5914e-01, 7.9043e-03, 1.2231e-02,
         2.2754e-01, 1.6416e-02, 1.1876e-02, 1.7510e-02, 3.3840e-01, 1.8072e-03,
         3.5154e-04, 3.4717e-04, 3.8318e-04, 6.2915e-04, 7.2895e-04, 8.2400e-04,
         1.0629e-03, 1.3867e-03, 5.7475e-04, 7.5882e-04, 9.3758e-04, 2.8053e-04],
        [1.7149e-04, 5.0561e-05, 2.1929e-06, 1.1317e-03, 1.1877e-06, 2.0934e-06,
         1.0553e-02, 1.7675e-06, 4.5773e-06, 2.1796e-06, 9.8806e-01, 4.7899e-09,
         6.6336e-07, 3.3186e-07, 3.2935e-07, 4.0715e-07, 4.1147e-07, 3.7682e-07,
         4.3787e-07, 8.9837e-07, 1.7923e-07, 1.7678e-06, 1.2857e-05, 6.8465e-07],
        [1.0508e-05, 3.4743e-06, 4.6870e-08, 3.8120e-04, 9.3855e-09, 5.2924e-08,
         3.0566e-03, 2.7807e-08, 1.1201e-07, 1.7496e-07, 9.9654e-01, 1.1696e-11,
         2.6010e-08, 1.2625e-08, 1.7537e-08, 2.6098e-08, 1.6074e-08, 9.3754e-09,
         1.1688e-08, 4.2408e-08, 9.5190e-09, 3.2413e-07, 2.9659e-06, 3.2108e-08],
        [1.9458e-03, 2.8252e-02, 3.9148e-04, 2.4568e-03, 3.6480e-04, 3.6632e-03,
         6.6316e-03, 1.0829e-03, 2.6483e-04, 9.1430e-04, 9.5341e-01, 6.5170e-06,
         4.1910e-05, 5.5306e-05, 4.4127e-05, 1.9787e-05, 3.0711e-05, 4.0539e-05,
         2.9807e-05, 4.6627e-05, 1.8199e-05, 6.1602e-05, 1.8459e-04, 3.9534e-05],
        [2.3346e-05, 2.3025e-04, 6.6342e-06, 9.7440e-06, 4.7095e-06, 1.5781e-04,
         3.6885e-05, 4.2335e-05, 5.7468e-07, 1.5478e-06, 9.9948e-01, 2.1849e-09,
         1.4737e-07, 1.6567e-07, 2.3395e-07, 1.0563e-07, 8.1612e-07, 1.8103e-06,
         8.1179e-07, 1.1271e-06, 1.3858e-07, 6.4910e-07, 3.2615e-06, 3.3964e-07],
        [1.2871e-03, 3.1355e-03, 5.8431e-06, 1.4571e-03, 2.8033e-05, 6.0734e-04,
         5.9160e-04, 1.0156e-04, 1.5693e-04, 2.3382e-05, 9.9251e-01, 4.4590e-08,
         3.7946e-06, 2.8240e-06, 2.4714e-06, 1.4381e-06, 3.5702e-06, 5.5475e-06,
         3.5290e-06, 1.3443e-05, 4.4258e-06, 1.7101e-05, 3.8267e-05, 3.0442e-06],
        [1.7301e-03, 7.6936e-02, 2.2392e-04, 4.2570e-03, 8.4958e-02, 2.6793e-04,
         3.9974e-03, 5.0231e-03, 6.1271e-01, 6.8372e-06, 2.0719e-01, 9.1594e-04,
         7.5850e-05, 1.0019e-04, 8.3039e-05, 5.9336e-05, 6.8828e-05, 9.8816e-05,
         6.0709e-05, 4.7183e-04, 5.1338e-04, 7.5312e-05, 1.1935e-04, 5.5234e-05],
        [8.4603e-05, 3.2398e-03, 3.9328e-06, 5.1563e-04, 1.3263e-04, 2.1148e-05,
         9.1606e-03, 1.2318e-05, 4.2726e-05, 7.9555e-05, 9.8646e-01, 1.2418e-07,
         1.7163e-06, 1.0168e-06, 6.0717e-07, 1.0121e-06, 2.6334e-06, 3.0241e-06,
         1.9109e-06, 4.3245e-06, 3.3401e-06, 3.0885e-05, 1.9430e-04, 5.9077e-06],
        [7.9048e-03, 2.3492e-01, 7.7534e-05, 9.6156e-02, 4.7931e-02, 8.1152e-04,
         1.0758e-02, 1.1800e-01, 9.4272e-02, 1.6358e-04, 3.7906e-01, 3.7232e-03,
         4.6190e-04, 2.7779e-04, 8.7715e-05, 6.4819e-05, 9.3710e-05, 1.3535e-04,
         1.8171e-04, 1.9018e-03, 1.2634e-03, 3.9480e-04, 9.3612e-04, 4.2582e-04],
        [2.2218e-02, 8.9862e-02, 3.6160e-03, 9.4319e-03, 1.0542e-01, 3.9693e-03,
         5.5532e-03, 4.8593e-02, 2.6865e-01, 6.9084e-05, 4.3166e-01, 5.3694e-03,
         8.7641e-04, 1.0177e-03, 6.8162e-04, 2.5425e-04, 2.8563e-04, 3.4366e-04,
         2.0688e-04, 7.0875e-04, 4.1897e-04, 1.0586e-04, 2.5030e-04, 4.3429e-04],
        [4.4966e-03, 1.9865e-02, 3.7699e-05, 9.9056e-04, 7.9092e-03, 3.7258e-04,
         4.6304e-04, 1.4475e-03, 5.0992e-01, 2.7054e-07, 4.5389e-01, 3.5430e-05,
         8.4039e-05, 5.7568e-05, 2.7272e-05, 6.6275e-06, 9.6975e-06, 2.5780e-05,
         1.5367e-05, 1.6744e-04, 1.0014e-04, 1.5782e-05, 3.4377e-05, 2.8773e-05],
        [5.3088e-03, 1.0592e-01, 5.0456e-04, 2.5137e-03, 1.3106e-01, 1.7145e-03,
         3.3702e-03, 4.0371e-03, 6.2267e-01, 2.9005e-05, 1.1985e-01, 5.1437e-04,
         7.3479e-05, 1.3013e-04, 1.2640e-04, 9.5820e-05, 1.8014e-04, 3.2298e-04,
         1.6177e-04, 6.5987e-04, 5.5676e-04, 8.1149e-05, 7.4907e-05, 3.7942e-05],
        [2.7496e-02, 1.9004e-01, 1.8134e-03, 1.4410e-02, 9.5382e-02, 2.1664e-03,
         4.3950e-02, 1.3132e-02, 3.1409e-01, 3.3461e-04, 2.8905e-01, 2.0364e-03,
         6.1260e-04, 8.4920e-04, 4.2517e-04, 2.1373e-04, 2.1415e-04, 3.3201e-04,
         3.6762e-04, 1.3615e-03, 9.0210e-04, 2.4184e-04, 3.4690e-04, 2.4572e-04],
        [2.5723e-05, 4.6458e-01, 6.2499e-07, 1.1877e-06, 1.0332e-01, 5.9065e-07,
         1.4935e-04, 6.3233e-05, 4.0666e-01, 6.8743e-09, 2.5055e-02, 2.6499e-06,
         7.6627e-06, 3.8626e-05, 3.9422e-06, 2.7858e-07, 2.9665e-07, 1.1049e-06,
         1.0493e-06, 3.8449e-05, 4.8190e-05, 8.7605e-07, 2.0218e-06, 1.2129e-06],
        [6.9856e-03, 1.2709e-01, 1.1437e-02, 3.5239e-03, 4.3952e-02, 8.9796e-03,
         6.7194e-03, 2.6811e-02, 4.8840e-01, 2.2110e-04, 2.6298e-01, 6.6461e-03,
         5.8478e-04, 8.3156e-04, 6.4815e-04, 1.5281e-04, 1.0460e-04, 1.8203e-04,
         1.8513e-04, 9.6580e-04, 1.3591e-03, 3.7557e-04, 4.4055e-04, 4.2379e-04],
        [1.0221e-03, 6.9503e-03, 1.6002e-04, 7.5357e-04, 2.4846e-03, 1.3012e-04,
         5.9110e-04, 6.8693e-04, 4.6920e-02, 3.7586e-07, 9.4004e-01, 1.1490e-05,
         2.3240e-05, 3.4033e-05, 4.6130e-05, 1.4531e-05, 1.2185e-05, 1.2849e-05,
         5.3007e-06, 3.7583e-05, 2.3812e-05, 8.2740e-06, 2.2788e-05, 1.1757e-05]]
y= [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.]]

In [14]:
# Initalize random X and y
x = torch.tensor(x, dtype=torch.float32, requires_grad=True).unsqueeze(0)
y = torch.tensor(y, dtype=torch.float32).unsqueeze(0)
print("Input x shape:", x.shape)
print("Input y shape:", y.shape)
# Create the sequences
# batch_size, len_x, len_y, dims = 8, 15, 12, 5
# x = torch.rand((batch_size, len_x, dims), requires_grad=True)
# y = torch.rand((batch_size, len_y, dims))

# Create the "criterion" object
sdtw = SoftDTW(use_cuda=False, gamma=0.1)

# Compute the loss value
loss = sdtw(x, y)  # Just like any torch.nn.xyzLoss()
print("Loss value:", loss)
print("Loss mean:", loss.mean())

# Aggregate and call backward()
loss.mean().backward()

Input x shape: torch.Size([1, 40, 24])
Input y shape: torch.Size([1, 23, 24])
Loss value: tensor([19.5312], grad_fn=<_SoftDTWBackward>)
Loss mean: tensor(19.5312, grad_fn=<MeanBackward0>)
